# 第7回：予測精度評価・モデル理解
* 前回はデータ準備（数値化）、予測モデル作成、精度確認を行いました。
* 精度がほぼ100%、AUCもほぼ1ととても良い指標値となりましたが、この値はどのくらい信用できるのでしょうか？
* 実は、機械学習では**学習に使用したデータ（学習用データ）に対して当てはまりが良い**ものの、**学習に使用していない新たなデータに対しては当てはまりが悪くなる**ことが多くあります。
* 予測モデル作成の目的は**新たなデータに対してうまく予測すること**なので、正しく予測精度を見積もるためには、**学習に使用していないデータに対する予測精度**を確認することが重要です。
* そこで今回は、予測モデルを作成する前に**データを学習用データと精度検証用データに分割**しておき、**精度検証用データに対する予測精度**を図ってみましょう。

# 1. データ読み込み
* まずはデータを読み込みましょう。
* 今回は前回演習で数値変換したデータ（data_ml.csv）を読み込みます。

In [ ]:
import pandas as pd             # 表形式のデータ操作ライブラリ
import matplotlib.pyplot as plt # グラフ描画ライブラリ
import seaborn as sns           # グラフ描画ライブラリ

from sklearn.tree import DecisionTreeClassifier      # 決定木による分類のためのライブラリ
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレストによる分類のためのライブラリ

from sklearn.metrics import roc_auc_score     # AUC値を算出するライブラリ
from sklearn.metrics import roc_curve         # ROC曲線を描画するためのライブラリ
from sklearn.metrics import accuracy_score    # 正解率を算出するライブラリ
from sklearn.metrics import confusion_matrix  # 混同行列を作成するライブラリ

from sklearn.model_selection import train_test_split # データを学習用と検証用に分割するライブラリ
from sklearn.model_selection import GridSearchCV     # グリッドサーチによるパラメータチューニングを実施するライブラリ

random_state = 1 # 乱数固定
sns.set() 

In [ ]:
# 警告（warning）を表示させない
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 前回の演習で作成済みのデータを読み込みます
data = pd.read_csv('data_ml.csv')

In [ ]:
print('データのサイズ：', data.shape) # データサイズ確認
display(data.head())                  # データが正しく読み込めたことを確認

# 2. データ準備
* 機械学習で予測モデルを作成するためのデータ準備を行います。

## 2-1. データを学習用データと検証用データに分割
* あらかじめデータを学習用と検証用に分けておき、学習用データでモデルを学習、検証用データで精度検証することで、未知のデータに対する性能を評価できます。

### 本演習での実施事項
* 本演習ではデータをランダムに8:2に分割しして、学習用データ、検証用データとして使用します（表を**行方向**に分割）
* sklearnで学習する際にはデータを説明変数と目的変数にも分割する必要があります（表を**列方向**に分割）

In [ ]:
X = data.drop(['y'],axis=1)
y = data.y

X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=0.2, random_state=random_state)  # 8:2に分割

#### 表示して確認

In [ ]:
print("-----X_train-----")
display(X_train.head())
print("-----y_train-----")
display(y_train.head())
print("-----X_test-----")
display(X_test.head())
print("-----y_test-----")
display(y_train.head())

#### データサイズも確認

In [ ]:
print('学習用データ：', X_train.shape)
print('検証用データ：', X_test.shape)

* 説明変数と目的変数に正しく分割できました。
* 学習用データは36168件、検証用データは9043件です。

In [ ]:
X = data.drop(['y'],axis=1)
y = data.y

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=random_state) 

# 3. 予測モデルの作成
* データの準備が終わったので、いよいよ予測モデルを作っていきましょう。
* 今回は決定木とランダムフォレストで予測モデルを作成します。
* 前回との変更部分は予測精度の評価部分で、予測モデルに使用していないX_testとy_testで評価します。

## 3-1. 学習・予測

### 決定木

In [ ]:
# モデルの定義
tree = DecisionTreeClassifier(random_state = random_state)     # この時点でtreeは未学習（常に同じ結果が出るようにrandom_stateで乱数固定）
# 学習
tree.fit(X_train, y_train)          # fit関数でtreeが学習データをもとに分岐を作り学習を行う
# 予測
y_pred_tree = tree.predict(X_test) # 学習済みのtreeを使ってX_testに対する予測を行う

### ランダムフォレスト

In [ ]:
# モデルの定義
forest = RandomForestClassifier(random_state = random_state)     # この時点では未学習（常に同じ結果が出るようにrandom_stateで乱数固定）
# 学習
forest.fit(X_train, y_train)          # fit関数でforestが学習データをもとに分岐を作り学習を行う
# 予測
y_pred_forest = forest.predict(X_test) # 学習済みのforestを使ってX_testに対する予測を行う

## 3-2. 精度評価
* 予測モデルが学習できたので、予測精度を評価してみましょう。
* 学習に使用していない検証用データに対してどのくらい予測が当たるのかを確認します。
* 前回の演習と同様に、今回も正解率とAUCで評価してみます。

### 正解率での評価

In [ ]:
ac_tree = accuracy_score(y_test, y_pred_tree)
ac_forest = accuracy_score(y_test, y_pred_forest)

print("決定木の正解率：{:.3f}".format(ac_tree))
print("ランダムフォレストの正解率：{:.3f}".format(ac_forest))

### 混同行列の表示

In [ ]:
cm_tree = confusion_matrix(y_test, y_pred_tree)
cm_forest = confusion_matrix(y_test, y_pred_forest)

print("---決定木の混同行列---")
print(cm_tree)
print("---ランダムフォレストの混同行列---")
print(cm_forest)

### ROCカーブの表示

In [ ]:
y_pred_tree = tree.predict_proba(X_test) # 0,1ではなく予測確率を出力

fpr, tpr, thresholds = roc_curve(y_test, y_pred_tree[:,1])
plt.figure(figsize=(5,5))
plt.rcParams["font.size"] = 18
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],color="gray",linestyle="dotted")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

In [ ]:
y_pred_forest = forest.predict_proba(X_test) # 0,1ではなく予測確率を出力

fpr, tpr, thresholds = roc_curve(y_test, y_pred_forest[:,1])
plt.rcParams["font.size"] = 18
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],color="gray",linestyle="dotted")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

### ROC AUCでの評価

In [ ]:
auc_tree = roc_auc_score(y_test, y_pred_tree[:,1])
auc_forest =  roc_auc_score(y_test, y_pred_forest[:,1])
print("決定木のAUC：{:.3f}".format(auc_tree))
print("ランダムフォレストのAUC：{:.3f}".format(auc_forest))

* AUCの値はビジネスケースやデータの形式にかかわらず同じ尺度で比較できるのがメリットです
    * 感覚論ですが一般的に0.7以上ならある程度予測ができており、0.9以上ならかなり精度の高い予測ができているといえます。
* ランダムフォレストのほうが良い予測ができているようです。

# 4. 作成したモデルの理解
* 予測精度の確認ができたところで、実際にどのようなモデルが作成されたのか確認してみましょう。
* 決定木やランダムフォレストでは、どの説明変数（特徴量）が予測に効果的なのかを確認できます。

## 4-1. ランダムフォレストの特徴量重要度（上位20個）

In [ ]:
feature_importances = pd.DataFrame(forest.feature_importances_,
                              index = X_test.columns,
                              columns=["importance"]).sort_values("importance",ascending=False)

feature_importances.head(20).plot.bar(figsize=(18,6), fontsize=24)

* balanceやageなどが予測に効いていることがわかります。

In [ ]:
# 各変数の特徴量重要度を表形式で表示
display(feature_importances)